In [ ]:
import os
import numpy as np
import lancet
import holoviews as hv

from featuremapper.analysis.raster import fft_power
from featuremapper.analysis.pinwheels import PinwheelAnalysis
from featuremapper.analysis.hypercolumns import PowerSpectrumAnalysis
from topo.analysis.command import *
from analysis import *

import topo
from topo.analysis import Collector
from topo.submodel.scal import ModelSCAL
from topo.submodel.gcal import ArraySpec

from topo.command import runscript  # In order to check the model files load correctly
from topo.misc.lancext import RunBatchCommand, topo_metadata

hv.notebook_extension()

# Experimental Setup

### Defining the parameter space

In [ ]:
batch_name = 'SCAL_spatial'

# Model options
laterals = True

# Measurements
rfs = True
isosuppression = True
sizetuning = True
frequencytuning = True
complexity = False
flankers = True

# Define times
times = [1000*i for i in range(21)]
print("Collection times start at %s and end at %s" % (min(times), max(times)))

# Define Args
constants = lancet.Args(area=4.0, aff_strength=2.4, exc_strength=1.6, inh_strength=1.8,
                        cortex_density=47, lgn_density=16, laterals=laterals, t_settle=16,)
batch_arguments = constants * lancet.Args(times=times) * lancet.List('dataset', ['Gaussian', 'natural', 'treeshrew'])

### Loading the model

In [ ]:
topo.sim.model = ModelSCAL(laterals=laterals, area=1.0)
scal = topo.sim.model.specification
topo.sim.model()

### Defining the measurements and analysis

In [ ]:
c = Collector()

# Projection activities
c.Activity.LGNOnAfferent =  c.collect(scal.projections.V1.LGNOnAfferent)
c.Activity.LGNOffAfferent = c.collect(scal.projections.V1.LGNOffAfferent)

# OR preference measurement
c.collect(measure_or_pref)
# Sheet activities
c.Activity.Retina =         c.collect(scal.sheets.Retina)
c.Activity.V1 =             c.collect(scal.sheets.V1)
# Connection fields
c.CFs.LGNOnAfferent =       c.collect(scal.projections.V1.LGNOnAfferent,  grid=True)
c.CFs.LGNOffAfferent =      c.collect(scal.projections.V1.LGNOffAfferent, grid=True)
c.CFs.LateralInhibitory =   c.collect(scal.projections.V1.LateralInhibitory, grid=True)
c.CFs.LateralExcitatory =   c.collect(scal.projections.V1.LateralExcitatory, grid=True)
if laterals:
    c.CFs.LRExcitatory =   c.collect(scal.projections.V1.LRExcitatory, rows=47, cols=47,
                                     grid=True, bounds=(-.5, -.5, .5, .5))

# Homeostatic threshold
c.HomeostaticThreshold.V1 = c.collect(ArraySpec('V1.output_fns[0].t'), 
                                                group='Homeostatic Threshold')

# OR preference measurement
c.collect(measure_or_pref, frequencies=[1.4, 1.6, 1.8])
c.collect(measure_response, durations=list(np.linspace(0, 1, 21)))
c.collect(measure_or_tuning_fullfield, contrasts=[5, 10, 20, 30, 50, 70, 100],
          frequencies=[1.4, 1.6, 1.8], times=[times[-1]])

if rfs:
    c.collect(measure_rfs, roi=(-.25, -.25, .25, .25), presentations=5000, scale=100, outputs=['V1'], times=[times[-1]])

# Times and coords for further measurements
coords=[(0,-0.1),(-0.1,0.0),(0,0),(0,0.1),(0.1,0.0)]
frequency=1.6

# Analysis
c.Pinwheels.V1   = c.analyze(c.ref.OrientationPreference.V1[:, -1.5:1.5, -1.5:1.5]
                             * c.ref.OrientationSelectivity.V1[:, -1.5:1.5, -1.5:1.5], PinwheelAnalysis)
c.FFTAnalysis.V1 = c.analyze(c.ref.OrientationPreference.V1[:, -1.5:1.5, -1.5:1.5], PowerSpectrumAnalysis)

# Measure position preference, requisite for other measurements
if sizetuning or frequencytuning or flankers or complexity or isosuppression:
    c.collect(measure_position_pref, x_range=(-0.4,0.4), y_range=(-0.4,0.4),
              size=0.1, outputs=['V1'], divisions=36, scale=2.0)

# Orientation Contrast Suppression
if isosuppression:
    c.analyze(c.ref.OrientationPreference.V1, measure_iso_suppression, output='V1',
              frequency=frequency, contrastcenter=70, contrastsurround=[10, 30, 70, 100],
              times=[times[-1]], mode='merge')

# Size Tuning Analysis
if sizetuning:
    c.analyze(c.ref.OrientationPreference.V1, measure_size_tuning, num_phase=8, outputs=['V1'],
              coords=coords, frequency=frequency, contrasts=[10, 100], times=[times[-1]], mode='merge')

# Measure PhaseTuning and Complexity
if complexity:
    c.analyze(c.ref.OrientationPreference.V1, measure_phase_tuning, outputs=['V1'], frequencies=[frequency],
              num_orientation=12, times=times, mode='merge')
    c.analyze(c.ref.PhaseTuning.V1, ComplexityAnalysis, times=[times[-1]], mode='merge')

# Measure flanker modulation
if flankers:
    c.collect(measure_flanker_ormodulation, coords=coords, outputs=['V1'], times=[times[-1]])
    c.collect(measure_flanker_xoffsetmodulation, coords=coords, outputs=['V1'], times=[times[-1]])
    c.collect(measure_flanker_yoffsetmodulation, coords=coords, outputs=['V1'], times=[times[-1]])

### Launching the jobs

In [ ]:
# Local or on cluster
QSUB = False
# Open diff in pager or not
SHOW_DIFF = True

ty_file = './scal_divisive.ty'
metadata = topo_metadata()
output_directory = os.path.join(os.getcwd(), 'data')

lancet.review_and_launch.output_directory = output_directory

qsub_options = dict(b='y',
                    pe=('memory-2G', '4'),   # Parallel environment allocation
                    v='OMP_NUM_THREADS=4',   # Must match slot allocation above.
                    #l='h_rt=05:59:00',       # Time resource allocation 
                    P='inf_ndtc')            # Project

@lancet.review_and_launch()
def launch():
    runbatch_cmd = RunBatchCommand(ty_file, c, metadata=batch_arguments.varying_keys)
    Launcher = lancet.QLauncher if QSUB else lancet.Launcher
    return Launcher(batch_name, batch_arguments, runbatch_cmd,  metadata=metadata(), 
                    **({'qsub_flag_options':qsub_options} if QSUB else {'max_concurrency': 3}))
launch()

# Monitoring Progress

In [ ]:
from holoviews.core.io import Unpickler
from analysis.progress import ProgressWidget, load_table
hv.notebook_extension('bokeh')

In [ ]:
path = './data/2016-03-19_2017-SCAL_spatial'
table = load_table(path)
data = Unpickler.collect(table, drop=['time', 'Index', 'tid', 'timestamps'])
ProgressWidget(path)

In [ ]:
cat ./data/2016-03-19_2017-SCAL_spatial/streams/*

# Results and Analysis

In [ ]:
hv.notebook_extension('matplotlib')

## Pinwheel Analysis

In [ ]:
orpref = data.OrientationPreference.V1.select(time=20000)().select(x=(-1.5, 1.5), y=(-1.5, 1.5))
PowerSpectrumAnalysis(orpref.reindex(['dataset'])).display('all').cols(2)

## DoG Size Tuning Fits

In [ ]:
%%output size=150 dpi=300 fig='svg'
%%opts Histogram (edgecolor='k' facecolor='white') [fontsize={'xlabel':15, 'ticks':14}]
%%opts Overlay [yaxis=None show_frame=False aspect=1.5]
%%opts Layout [sublabel_position=(-0.15, 0.85) aspect_weight=1 hspace=0.3]
sizetuning = data.SizeTuning.V1.select(dataset='Gaussian')()
df = sizetuning.last.data
peak_mean = df['Peak Size'].mean()
peak_median = df['Peak Size'].median()
surr_mean = df['Suppression Size'].mean()
surr_median = df['Suppression Size'].median()
peak_arrow = hv.Arrow(peak_median, 0, '', 'v')
surr_arrow = hv.Arrow(surr_median, 0, '', 'v')

size_tuning = ((hv.Table(df).hist(dimension='Peak Size', adjoin=False, num_bins=11)\
 .clone(kdims=[hv.Dimension('$r_c$', unit='$\circ$')]) * peak_arrow).relabel('Size tuning center') +
(hv.Table(df).hist(dimension='Suppression Size', adjoin=False, num_bins=11)\
 .clone(kdims=[hv.Dimension('$r_s$', unit='$\circ$')]) * surr_arrow).relabel('Size tuning surround'))
size_tuning

In [ ]:
hv.Store.dump(size_tuning, open('SCAL_SizeTuning.pkl', 'wb'))

## Suppression Index

In [ ]:
import scipy.stats as ss

In [ ]:
%%output dpi=120 size=120
%%opts Overlay [aspect=1.5 show_frame=False]
si_dist = (hv.Table(df).hist(dimension='SI', adjoin=False).relabel(group='Suppression Index') *
           hv.Text(0.6, 3, 'Mean SI: $%.3f \pm %.3f$' % (df.SI.mean(), ss.sem(df.SI)), fontsize=10))
si_dist

## Area summation/size tuning curves

In [ ]:
sizeresponse = data.SizeResponse.V1()
size_curves = hv.GridSpace(kdims=['X', 'Y'])
for (x, y), responses in sizeresponse.groupby(['X', 'Y']).items():
    sampled = responses.sample((5, 5), bounds=(x-0.05, y-0.05, x+0.05, y+0.05))
    size_curves[x, y] = sampled.to.curve(['Size'], ['Response']).overlay('Contrast').grid(['x', 'y'])

In [ ]:
size_grid = size_curves.values()[1]

In [ ]:
%%opts GridSpace [fig_size=200]
size_grid

In [ ]:
%%output dpi=120 size=120
%%opts NdOverlay [aspect=1.5] Layout [aspect_weight=0.5]
size_grid[0.03, -.1].relabel(group='Size Tuning') + size_grid[0.01, -.14].relabel(group='Size Tuning')

## Contrast dependent size tuning shift

In [ ]:
import scipy.stats as ss
contrast_shift = data.ContrastShift.V1()

In [ ]:
test = contrast_shift.hist(dimension='CSS', adjoin=False, bin_range=(0.8, 1.5))

In [ ]:
%%output dpi=120 size=120
%%opts Overlay [aspect=1.5 show_frame=False]
css_dist = (contrast_shift.hist(dimension='CSS', adjoin=False, bin_range=(0.8, 1.5)).values()[0] *
            hv.Text(1.3, 8, 'Mean CSS: $%.3f \pm %.3f$' % (contrast_shift.last.data.CSS.mean(),
                                            ss.sem(contrast_shift.last.data.CSS)), fontsize=10))
css_dist

## Orientation contrast suppression index

In [ ]:
ocsi = data.OCSI_Analysis.V1()

In [ ]:
%%output dpi=120 size=120
%%opts Overlay [aspect=1.5 show_frame=False]
ocsi_high = ocsi['Gaussian', 20000, 100]
ocsi_high_df = ocsi_high.data.dropna()
ocsi_dist = (ocsi_high.hist(adjoin=False, bin_range=(0, 1.2), normed=False) *
             hv.Text(0.5, 20, 'Mean OCSI: $%.3f \pm %.3f$' % (ocsi_high_df[ocsi_high_df.OCSI > -6].OCSI.mean(),
                                                             ss.sem(ocsi_high_df[ocsi_high_df.OCSI > -6].OCSI)),
                     fontsize=10))
ocsi_dist

In [ ]:
%%output widgets='live'
%%opts Image (cmap='RdBu_r')
orcontrast.select(x=(-.1, 0.1), y=(-.1, 0.1))

In [ ]:
orcontrast = data.OrientationContrastResponse.V1.select(dataset='natural')()
ocsi_curves = hv.GridSpace(kdims=['X', 'Y'])
for (x, y), responses in orcontrast.groupby(['X', 'Y']).items():
    ocsi_curves[x, y] = responses.sample((5, 5), bounds=(x-0.05, y-0.05, x+0.05, y+0.05)).to.curve(['OrientationSurround'], ['Response']).overlay('ContrastSurround').grid(['x', 'y'])
ocsi_grid = ocsi_curves.values()[0]

In [ ]:
%%opts Curve [xticks=5]
ocsi_grid.map(lambda x: hv.Scatter(x)[-np.pi/2: np.pi/2.], [hv.Curve])[0,0]

In [ ]:
%%output dpi=120 size=120
%%opts Histogram {+axiswise} (facecolor='white') Text {+axiswise}
surround_tuning = si_dist + css_dist + ocsi_dist
surround_tuning

In [ ]:
hv.Store.dump(surround_tuning, open('SCAL_Surround.pkl', 'wb'))

### Orientation Tuning

In [ ]:
%%output size=150
%%opts NdOverlay [xaxis=None]
ortuning = data.OrientationTuning.V1()
ortuning_samples = ortuning.sample((8, 8))
tuning_grid = ortuning_samples.to.curve('Orientation', 'Response').overlay('Contrast').grid(['x', 'y'])
tuning_grid

In [ ]:
tuning_overlay = tuning_grid[1.76, 1.24].last

In [ ]:
normalized_tuning = tuning_overlay.clone(shared_data=False)
for key, curve in tuning_overlay.items():
    curve_data = curve.columns()
    max_r = curve.range('Response')[1]
    max_r = max_r if max_r else 1
    curve_data['Normalized Response'] = curve_data['Response']/max_r
    normalized_tuning[key] = curve.clone(curve_data, vdims=['Normalized Response'])

In [ ]:
%%opts NdOverlay [aspect=1.5] Layout [fig_size=150]
tuning_overlay + normalized_tuning(style={'Curve': dict(color=hv.core.options.Palette('gray', reverse=True))})

In [ ]:
tuning_overlay[10].data

## Weight Distrubtion Plots

In [ ]:
from topo.analysis.weights import WeightDistribution, WeightIsotropy

latinh = data.CFs.LateralInhibitory.select(time=20000)()
orpref = data.OrientationPreference.V1.select(time=20000)()
xpref = data.XPreference.V1.select(time=20000)()
ypref = data.YPreference.V1.select(time=20000)()

tree = hv.Layout()
tree.OrientationPreference.V1 = orpref.last
tree.XPreference.V1 = xpref.last
tree.YPreference.V1 = ypref.last
tree.CFs.LateralInhibitory = latinh

weight_orientation = WeightDistribution(tree, projections=[('V1', 'LateralInhibitory')])
weight_isotropy = WeightIsotropy(tree, projections=[('V1', 'LateralInhibitory')], num_bins=10)
weight_orientation + weight_isotropy

In [ ]:
tree = hv.Layout()
tree.CFs.LRExcitatory = data.CFs.LRExcitatory.select(time=20000)()[-1:1, -1:1]
tree.OrientationPreference.V1 = data.OrientationPreference.V1.select(time=20000)()
vonMises = analysis.CFvonMisesFit(tree, sheet='V1', projection='LRExcitatory', fit_aspect=True, threshold=0.7)

In [ ]:
vonMises.Results.Table.dframe()

In [ ]:
%%output size=150
(vonMises.Preprocessed.CFs + hv.Empty() +\
vonMises.NaiveFit.CFs + vonMises.NaiveFit.Error +\
vonMises.VonMisesFit.CFs + vonMises.VonMisesFit.Error).cols(2)

In [ ]:
%%opts Distribution (kde_kws=dict(bw=0.5, cut=0) hist=True) {+axiswise} Overlay [aspect=2] Layout [fig_size=150] 
%%opts VLine (color='k') {+axiswise} Text {+axiswise}
lat_fit_table = table.select(dataset='Gaussian', Model='vonMises', mu1=(0, 10))
txt = '$\sigma_{{{0}}} = {1:.5g} mm$'
mean_mu1 = np.mean(lat_fit_table.data.mu1*3)
mean_mu2 = np.mean(lat_fit_table.data.mu2*3)
hv.Distribution(lat_fit_table.data.mu1*3) * hv.VLine(mean_mu1) * hv.Text(mean_mu1, 0.8, txt.format('LR', mean_mu1)) +\
hv.Distribution(lat_fit_table.data.mu2*3) * hv.VLine(mean_mu2) *  hv.Text(mean_mu2, 6, txt.format('LOC', mean_mu2))

In [ ]:
%%opts BoxWhisker [aspect=1 fig_size=100]
vonMises.Results.Table.table().to.box(['Model'], 'MSE', ['dataset']).layout()

In [ ]:
%%opts BoxWhisker [aspect=1 fig_size=100]
vonMises.Results.Table.table().select(Model='vonMises').to.box(['dataset'], 'aspect', [])

In [ ]:
%%opts BoxWhisker [aspect=1 fig_size=100]
vonMises.Results.Table.table().to.box(['Model'], 'r2', ['dataset']).layout()

# RF Tuning

In [ ]:
rfs = data.Retina_Reverse_Correlation.V1()
orpref = data.OrientationPreference.V1.select(Time=20000)()

In [ ]:
rffit = RFGaborFit(rfs, orpref, roi_radius=1, max_iterations=10000)
fit_table = rffit.RFGaborFit.RF_Fit_Values.table()
fit_table.data['Time'] = fit_table.data.Time.astype(np.float)
fit_table.data['Duration'] = fit_table.data.Duration.astype(np.float)

In [ ]:
%%opts Image {+framewise +axiswise} GridSpace [normalize=True]
(rffit.RFGaborFit.RF_Normed + rffit.RFGaborFit.RF_Fit).cols(2).display('all')

In [ ]:
sliced = fit_table.select(f=(0,10), nx=(0, 2), ny=(0,2))
hv.Layout([sliced.to.box(['dataset'], [value], []) for value in ['residual', 'f', 'nx']])

In [ ]:
%%opts NdOverlay [fig_size=250]
fit_table.to.scatter(['nx'], ['ny'], ['dataset'])[:, 0:1, 0:1].overlay()